# Downloading data

In [ ]:
!pip install wget
!pip install jsonlines

!gdown --id 1zME0dkPL90tqMfzLzIl9ZT7GZcY8meG0

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=a8cacf35ebd508768ee8573aacf70a58a3cbe7ae8bffc32cad007a05a155eb31
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [ ]:
from MultiNLI import download_and_unzip_multinli, pandas_decorator

download_and_unzip_multinli_pd = (download_and_unzip_multinli)
train, matched_val, mismatched_val = download_and_unzip_multinli_pd('', '')

Unzipping multinli_1.0.zip to 
Reading multinli_1.0/multinli_1.0_train.jsonl
Reading multinli_1.0/multinli_1.0_dev_matched.jsonl
Reading multinli_1.0/multinli_1.0_dev_mismatched.jsonl
Done


In [ ]:
missed_contradictions = 0
total_guesses = 0

for i in range(len(matched_val)):
    gold_label = matched_val[i]['gold_label']
    annotator_labels = matched_val[i]['annotator_labels']
    if gold_label == 'contradiction':
        batch_missed_contr = 0
        for label in annotator_labels:
            if label != gold_label:
                batch_missed_contr += 1
        if batch_missed_contr >= 2: #тут тонкий момент: я считаю, что эксперт ошибся, если 2 из 5 annotator_labels не contradiction. ниже можно посмотреть, что будет если искать ВСЕ недопредсказанные contradiction'ы
            missed_contradictions += 1 
    for label in annotator_labels:
        total_guesses += 1

print("вероятность эксперат ошибиться: {0}".format(missed_contradictions / total_guesses))

вероятность эксперат ошибиться: 0.0073


In [ ]:
missed_contradictions = 0
total_guesses = 0

for i in range(len(matched_val)):
    gold_label = matched_val[i]['gold_label']
    annotator_labels = matched_val[i]['annotator_labels']
    if gold_label == 'contradiction':
        batch_missed_contr = 0
        for label in annotator_labels:
            if label != gold_label:
                batch_missed_contr += 1
        if batch_missed_contr >= 1: #тут тонкий момент: я считаю, что эксперт ошибся, если 2 из 5 annotator_labels не contradiction. ниже можно посмотреть, что будет если искать ВСЕ недопредсказанные contradiction'ы
            missed_contradictions += 1 
    for label in annotator_labels:
        total_guesses += 1

print("вероятность эксперат ошибиться: {0}".format(missed_contradictions / total_guesses))

вероятность эксперат ошибиться: 0.02054


In [ ]:
import pickle
!gdown --id 1wbDrzVPpsuJMT1jZin14QMpvmbviIYBW
with open('deberta-v2-xxlarge preds.pickle', 'rb') as f:
    preds_and_labels = pickle.load(f)

Downloading...
From: https://drive.google.com/uc?id=1wbDrzVPpsuJMT1jZin14QMpvmbviIYBW
To: /content/deberta-v2-xxlarge preds.pickle
7.03MB [00:00, 60.3MB/s]


In [ ]:
gold_label = [pred.item() for pred in preds_and_labels['gold_label']]

In [ ]:
threshold = 0.99
preds = [(pred.argmax(-1).item(), label.item()) for (pred, label) in zip(preds_and_labels['softmaxed_logits'], preds_and_labels['gold_label']) if torch.max(pred) > threshold]
y_preds = [pred[0] for pred in preds]
y_true = [pred[1] for pred in preds]

contradictions = 0
missed_contradictions = 0
for y_pred, y_true_el in zip(y_preds, y_true):
    if y_true_el == 0:
        contradictions += 1
        if y_pred != 0:
            missed_contradictions += 1

missed_contradictions / contradictions
print("вероятность модели недопредсказать contradiction при уверенности больше {1}: {0}\nТогда доля датасета, с которой работает модель: {2}".format(missed_contradictions / contradictions, threshold, len(preds) / len(preds_and_labels['gold_label'])))

вероятность модели недопредсказать contradiction при уверенности больше 0.99: 0.006247211066488175
Тогда доля датасета, с которой работает модель: 0.5023942944472746


In [ ]:
threshold = 0.92
preds = [(pred.argmax(-1).item(), label.item()) for (pred, label) in zip(preds_and_labels['softmaxed_logits'], preds_and_labels['gold_label']) if torch.max(pred) > threshold]
y_preds = [pred[0] for pred in preds]
y_true = [pred[1] for pred in preds]

contradictions = 0
missed_contradictions = 0
for y_pred, y_true_el in zip(y_preds, y_true):
    if y_true_el == 0:
        contradictions += 1
        if y_pred != 0:
            missed_contradictions += 1

missed_contradictions / contradictions
print("вероятность модели недопредсказать contradiction при уверенности больше {1}: {0}\nТогда доля датасета, с которой работает модель: {2}".format(missed_contradictions / contradictions, threshold, len(preds) / len(preds_and_labels['gold_label'])))

вероятность модели недопредсказать contradiction при уверенности больше 0.92: 0.020313613684960798
Тогда доля датасета, с которой работает модель: 0.7904228222109017
